# LiFePO₄ Battery Bank Analysis Notebook

This notebook reproduces the key analyses from the Architectural Immunity Study.

## Requirements
```bash
pip install pandas numpy matplotlib scipy
```

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from datetime import datetime

# Configure plots
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['font.size'] = 12

print("Libraries loaded successfully!")

## 1. Load Data

In [ ]:
# Load voltage data
df = pd.read_csv('../Data/combined_output.csv', parse_dates=['Timestamp'])

# Calculate derived columns
df['Voltage_Avg'] = (df['Voltage_Min'] + df['Voltage_Max']) / 2
df['Spread_mV'] = (df['Voltage_Max'] - df['Voltage_Min']) * 1000

print(f"Data loaded: {len(df)} records")
print(f"Date range: {df['Timestamp'].min()} to {df['Timestamp'].max()}")
print(f"Duration: {(df['Timestamp'].max() - df['Timestamp'].min()).days} days")
df.head()

## 2. Descriptive Statistics

In [ ]:
print("=" * 50)
print("VOLTAGE STATISTICS")
print("=" * 50)
print(f"Mean Voltage:     {df['Voltage_Avg'].mean():.4f} V")
print(f"Std Deviation:    {df['Voltage_Avg'].std():.4f} V ({df['Voltage_Avg'].std()*1000:.2f} mV)")
print(f"Min Voltage:      {df['Voltage_Min'].min():.4f} V")
print(f"Max Voltage:      {df['Voltage_Max'].max():.4f} V")
print()
print("=" * 50)
print("SPREAD STATISTICS (Cell Balance Indicator)")
print("=" * 50)
print(f"Mean Spread:      {df['Spread_mV'].mean():.2f} mV")
print(f"Max Spread:       {df['Spread_mV'].max():.2f} mV")
print(f"Min Spread:       {df['Spread_mV'].min():.2f} mV")

## 3. Voltage Timeline Plot

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6))

ax.fill_between(df['Timestamp'], df['Voltage_Min'], df['Voltage_Max'], 
                alpha=0.3, color='blue', label='Min-Max Range')
ax.plot(df['Timestamp'], df['Voltage_Avg'], color='blue', linewidth=0.5, label='Average')

ax.set_xlabel('Date')
ax.set_ylabel('Voltage (V)')
ax.set_title('Battery Bank Voltage Over Time')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Spread Analysis (Cell Balance)

In [ ]:
fig, ax = plt.subplots(figsize=(14, 5))

ax.plot(df['Timestamp'], df['Spread_mV'], color='green', linewidth=0.5)
ax.axhline(y=df['Spread_mV'].mean(), color='red', linestyle='--', label=f'Mean: {df["Spread_mV"].mean():.1f} mV')

ax.set_xlabel('Date')
ax.set_ylabel('Spread (mV)')
ax.set_title('Voltage Spread: Max - Min (Cell Balance Indicator)')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Drift Analysis (Linear Regression)

In [ ]:
# Convert timestamp to numeric (days since start)
df['Days'] = (df['Timestamp'] - df['Timestamp'].min()).dt.total_seconds() / 86400

# Linear regression
slope, intercept, r_value, p_value, std_err = stats.linregress(df['Days'], df['Voltage_Avg'])

print("=" * 50)
print("DRIFT ANALYSIS (Linear Regression)")
print("=" * 50)
print(f"Slope:            {slope*1000:.4f} mV/day")
print(f"R-squared:        {r_value**2:.4f}")
print(f"P-value:          {p_value:.2e}")
print(f"Std Error:        {std_err*1000:.4f} mV/day")
print()
if p_value < 0.05:
    print("⚠️ Statistically significant drift detected")
else:
    print("✅ No statistically significant drift")

# Plot
fig, ax = plt.subplots(figsize=(14, 6))
ax.scatter(df['Days'], df['Voltage_Avg'], alpha=0.3, s=1)
ax.plot(df['Days'], slope * df['Days'] + intercept, color='red', 
        label=f'Trend: {slope*1000:.3f} mV/day')

ax.set_xlabel('Days Since Start')
ax.set_ylabel('Average Voltage (V)')
ax.set_title('Voltage Drift Analysis')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Distribution Analysis

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Voltage distribution
axes[0].hist(df['Voltage_Avg'], bins=50, edgecolor='black', alpha=0.7)
axes[0].axvline(df['Voltage_Avg'].mean(), color='red', linestyle='--', label='Mean')
axes[0].set_xlabel('Voltage (V)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Voltage Distribution')
axes[0].legend()

# Spread distribution
axes[1].hist(df['Spread_mV'], bins=50, edgecolor='black', alpha=0.7, color='green')
axes[1].axvline(df['Spread_mV'].mean(), color='red', linestyle='--', label='Mean')
axes[1].set_xlabel('Spread (mV)')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Spread Distribution')
axes[1].legend()

plt.tight_layout()
plt.show()

## 7. Key Findings Summary

In [ ]:
# Calculate key metrics
duration_days = (df['Timestamp'].max() - df['Timestamp'].min()).days
total_drift_mV = slope * duration_days * 1000

print("=" * 60)
print("ARCHITECTURAL IMMUNITY STUDY - KEY FINDINGS")
print("=" * 60)
print()
print(f"📅 Study Duration:        {duration_days} days")
print(f"📊 Data Points:           {len(df):,} hourly samples")
print()
print("VOLTAGE STABILITY")
print(f"   Mean Voltage:          {df['Voltage_Avg'].mean():.4f} V")
print(f"   Std Deviation:         {df['Voltage_Avg'].std()*1000:.2f} mV")
print(f"   Total Drift:           {total_drift_mV:.1f} mV over {duration_days} days")
print(f"   Drift Rate:            {slope*1000:.3f} mV/day")
print()
print("CELL BALANCE")
print(f"   Mean Spread:           {df['Spread_mV'].mean():.2f} mV")
print(f"   Max Spread:            {df['Spread_mV'].max():.2f} mV")
print()
print("CONCLUSION")
if df['Spread_mV'].mean() < 20 and abs(slope*1000) < 2:
    print("   ✅ ARCHITECTURAL IMMUNITY CONFIRMED")
    print("   Mixed-brand cells achieved monolithic behavior")
else:
    print("   ⚠️ Further investigation recommended")
print()
print("=" * 60)

## 8. Temperature Correlation (if data available)

In [ ]:
try:
    temp_df = pd.read_csv('../Data/Combined_Temperature_Data.csv', parse_dates=['Timestamp'])
    
    # Merge on nearest timestamp
    merged = pd.merge_asof(
        df.sort_values('Timestamp'), 
        temp_df.sort_values('Timestamp'),
        on='Timestamp',
        direction='nearest'
    )
    
    # Correlation
    if 'Temperature' in merged.columns:
        corr = merged['Voltage_Avg'].corr(merged['Temperature'])
        print(f"Temperature-Voltage Correlation: r = {corr:.3f}")
        
        # Calculate thermal coefficient
        slope_temp, _, _, _, _ = stats.linregress(merged['Temperature'], merged['Voltage_Avg'])
        print(f"Thermal Coefficient: {slope_temp*1000:.3f} mV/°F")
    else:
        print("Temperature column not found in merged data")
        
except FileNotFoundError:
    print("Temperature data file not found - skipping correlation analysis")
except Exception as e:
    print(f"Error loading temperature data: {e}")

---

## How to Use This Notebook

1. **Clone the repository:**
   ```bash
   git clone https://github.com/wkcollis1-eng/Lifepo4-Battery-Banks.git
   cd Lifepo4-Battery-Banks/notebooks
   ```

2. **Install dependencies:**
   ```bash
   pip install pandas numpy matplotlib scipy jupyter
   ```

3. **Run the notebook:**
   ```bash
   jupyter notebook analysis_notebook.ipynb
   ```

4. **Modify for your data:** Replace `combined_output.csv` with your own data file.

---

*Last updated: January 10, 2026*